## **Łączenie z IBM Cloudant**

#### Import biblioteki:
<ul>
    <li>Cloudant - zapewnia interfejs do nawiązywania połączenia z usługą Cloudant, wykonywania operacji na bazach danych, tworzenia, modyfikowania i usuwania dokumentów itp.</li>
    <li> 
<ul>

In [2]:
from cloudant.client import Cloudant
from cloudant.design_document import DesignDocument
from cloudant.query import QueryResult
from cloudant.view import ViewResult
from cloudant.error import CloudantException
from cloudant.result import Result
from cloudant.search import SearchResult

#### Wprowadzenie danych do konta:

In [3]:
#Nazwa użytkownika
serviceUsername = "apikey-v2-58B528DF5397465BB6673E1B79482A8C"

#Hasło
servicePassword = "49c0c343d225623956157d94b25d574586f26d1211e8e589646b4713d5de4801"

#URL bazy danych w IBM Cloudant
serviceURL = "https://353466e8-47eb-45ce-b125-4a4e1b5a4f7e-bluemix.cloudant.com"

#### Połączenie z chmurą
W celu połączenia się z IBM Cloudant należy utworzyć klienta, podając dane do konta IBM Cloud.

In [ ]:
client = Cloudant(serviceUsername, servicePassword, url=serviceURL)
client.connect()

## **Utworzenie nowej bazy danych**

In [ ]:
my_database = client.create_database('database1')

In [ ]:
my_database = client['database1']

#### Sprawdzenie, czy baza danych została poprawnie utworzona:

In [ ]:
if my_database.exists():
    print('SUCCESS!!')

#### Utworzenie partitioned database

Partitioned databases to bazy danych, które wprowadzają możliwość tworzenia logicznych grup dokumentów, zwanych partycjami, poprzez przypisanie klucza partycji do każdego dokumentu. Aby móc utworzyć taką bazę danych, należy w ustawieniach włączyć opcję `partitions`.

In [ ]:
db = client.create_database('mydb', partitioned=True)

## **Utworzenie dokumentu w partitioned database**
Identyfikator dokumentu składa się zarówno z klucza partycji, jak i klucza dokumentu w formacie `<partitionkey>:<documentkey>`, gdzie:
<ul>
<li>Partition key (string) - musi być niepusty, nie może zawierać dwukropków (ponieważ są one używane jako separator kluczy partycji) ani zaczynać się od podkreślenia.</li>
<li>Document key (string) - musi być niepusty, nie może zaczynać się od podkreślenia.</li>
</ul>

Należy pamiętać, że dokumenty `_design` i `_local` nie mogą zawierać klucza partycji, ponieważ są to globalne definicje i mają zastosowanie do całej bazy danych. 

Dokumenty `_design` są używane do definiowania widoków, które pomagają w indeksowaniu i przeszukiwaniu danych w bazie Cloudant. Widoki pozwalają na definiowanie funkcji mapowania i redukcji, które są stosowane do danych przechowywanych w bazie danych. Mogą być używane do wykonywania zaawansowanych zapytań i agregacji danych. 

Dokumenty `_local` z kolei przechowują lokalne ustawienia i konfiguracje bazy danych.

In [ ]:
partition_key = 'CategoryA'
document_key = 'product123'

db.create_document({
    '_id': ':'.join((partition_key, document_key)),
    'name': 'Example Product',
    'price': 19.99,
    'description': 'This is an example product in Category A.',
    'in_stock': True
})

#### Pobieranie dokumentu z bazy danych o określonych partition key i document key

In [ ]:
doc = db[':'.join((partition_key, document_key))]

#### Tworzenie dokumentu _design

In [ ]:
ddoc = DesignDocument(db, document_id='view', partitioned=True)
ddoc.add_view('myview','function(doc) { emit(doc.foo, doc.bar); }')
ddoc.save()

#### Tworzenie indeksu zapytań

In [ ]:
index = db.create_query_index(
    design_document_id='query',
    index_name='foo-index',
    fields=['foo'],
    partitioned=True
)
index.create()

#### Wykonywanie zapytań w bazie danych Cloudant

Query:

In [ ]:
results = self.db.get_partitioned_query_result(
    partition_key, selector={'foo': {'$eq': 'bar'}})

for result in results:
    get_partitioned_query_result()

Views:

In [ ]:
results = self.db.get_partitioned_view_result(
    partition_key, view_ddoc['_id'], 'view1')

for result in results:
    get_partitioned_view_result()

## **Dokumenty**

In [ ]:
data = {
    '_id': 'julia30', #Ustawienie _id w tym miejscu jest opcjonalne
    'name': 'Julia',
    'age': 30,
    'pets': ['cat', 'dog', 'frog']
    }

#Utworzenie dokumentu z wykorzystaniem Database API
my_document = my_database.create_document(data)

#Sprawdzenie, czy dokument został poprawnie utworzony
if my_document.exists():
    print('SUCCESS!!')

## **Uzyskiwanie dokumentu z bazy**

In [ ]:
my_document = my_database['julia30']

#Wyświetlenie dokumentu
print(my_document)

## **Uzyskiwanie wszystkich dokumentów znajdujących się w bazie**

In [ ]:
for document in my_database:
    print(document)

## **Updatowanie danych w dokumencie**

In [ ]:
#Wyciągnięcie dokumentu z bazy
my_document = my_database['julia30']

#Updatowanie wybranych danych tak jak w słowniku
my_document['name'] = 'Jules'
my_document['age'] = 6

#Zapisujemy zmiany w dokumencie
my_document.save()

## **Usuwanie dokumentu**

In [ ]:
my_document = my_database['julia30']

my_document.delete()

## **Uzyskiwanie dokumentów wraz z ich zawartością**

Klasa `Result` w bibliotece Cloudant dostarcza interfejsu umożliwiającego dostęp, wycięcie i iterację po wynikach z naszych dokumentów.

In [ ]:
from cloudant.result import Result, ResultByKey

# Retrieve Result wrapped document content.
# Note: The include_docs parameter is optional and is used to illustrate that view query
# parameters can be used to customize the result collection.
result_collection = Result(my_database.all_docs, include_docs=True)

# Get the result at a given location in the result collection
# Note: Valid result collection indexing starts at 0
result = result_collection[0]                   # result is the 1st in the collection
result = result_collection[9]                   # result is the 10th in the collection

# Get the result for matching a key
result = result_collection['julia30']           # result is all that match key 'julia30'

# If your key is an integer then use the ResultByKey class to differentiate your integer
# key from an indexed location within the result collection which is also an integer.
result = result_collection[ResultByKey(9)]      # result is all that match key 9

# Slice by key values
result = result_collection['julia30': 'ruby99'] # result is between and including keys
result = result_collection['julia30': ]         # result is after and including key
result = result_collection[: 'ruby99']          # result is up to and including key

# Slice by index values
result = result_collection[100: 200]            # result is between 100 to 200, including 200th
result = result_collection[: 200]               # result is up to and including the 200th
result = result_collection[100: ]               # result is after the 100th

# Iterate over the result collection
for result in result_collection:
    print(result)

## **Usuwanie bazy danych z IBM Cloudant**

In [ ]:
client.delete_database('my_database')

## **Rozłączenie z IBM Cloudant**

In [ ]:
client.disconnect()